# Retrieving Data

In [2]:
import numpy as np
import pandas as pd
from datetime import time
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time

# Source Data

In [6]:
# Define the function to get the sector for each stock
def get_sector(ticker):
    try:
        stock_info = yf.Ticker(ticker).info
        return stock_info.get('sector', 'N/A')  # Returns 'N/A' if sector information is not available
    except Exception as e:
        #print(f"Could not fetch sector for {ticker}: {e}")
        return 'N/A'
    

# get ticker symbols
tickers = pd.read_csv('data/tickers.csv')
tickers = tickers['Symbol'].tolist()

sectors = []

# Loop through each ticker and fetch the sector
for ticker in tickers:
    print(f"Fetching sector for {ticker}")
    sector = get_sector(ticker)
    sectors.append(sector)
   # time.sleep(0.5)  # Delay to avoid hitting API rate limits

Fetching sector for MSFT
Fetching sector for AAPL
Fetching sector for NVDA
Fetching sector for AMZN
Fetching sector for GOOGL
Fetching sector for META
Fetching sector for GOOG
Fetching sector for BRK.B
Fetching sector for AVGO
Fetching sector for LLY
Fetching sector for JPM
Fetching sector for XOM
Fetching sector for TSLA
Fetching sector for UNH
Fetching sector for V
Fetching sector for MA
Fetching sector for PG
Fetching sector for JNJ
Fetching sector for HD
Fetching sector for MRK
Fetching sector for COST
Fetching sector for CVX
Fetching sector for ABBV
Fetching sector for CRM
Fetching sector for BAC
Fetching sector for WMT
Fetching sector for AMD
Fetching sector for NFLX
Fetching sector for PEP
Fetching sector for KO
Fetching sector for TMO
Fetching sector for ADBE
Fetching sector for WFC
Fetching sector for LIN
Fetching sector for DIS
Fetching sector for MCD
Fetching sector for CSCO
Fetching sector for ACN
Fetching sector for ORCL
Fetching sector for ABT
Fetching sector for QCOM
Fet

In [8]:
# get tickers
tickers = pd.read_csv('data/tickers.csv')
tickers['Sector'] = sectors
tickers.to_csv('data/tickers.csv', index=False)

   #                Company Symbol Portfolio%                  Sector
0  1         Microsoft Corp   MSFT      7.05%              Technology
1  2             Apple Inc.   AAPL      5.74%              Technology
2  3            Nvidia Corp   NVDA      5.06%              Technology
3  4         Amazon.com Inc   AMZN      3.83%       Consumer Cyclical
4  5  Alphabet Inc. Class A  GOOGL      2.37%  Communication Services


In [36]:
#get ticker symbols
ticker_symbols = tickers['Symbol'].tolist()

# Split tickers into batches of 100 for we don't hit the API rate limit
ticker_batches = [ticker_symbols[i:i + 100] for i in range(0, len(ticker_symbols), 100)]

# Start and end dates
start_date = '2005-01-01'
end_date = '2024-01-01'

# Create an empty DataFrame to store all data
all_data = pd.DataFrame()

# Loop through each batch of tickers and fetch the historical data
for batch in ticker_batches:
    for ticker in batch:
        print(f"Downloading data for {ticker}")
        data = yf.download(ticker, start=start_date, end=end_date, interval="1wk")
        data.dropna(inplace=True)  # Clean data by dropping NAs
        data['Ticker'] = ticker  # Add a column to identify the stock
        all_data = pd.concat([all_data, data], axis=0)
    time.sleep(10)  # sleep tim to avoid hitting API rate limits

# Save the final result to a CSV file
all_data.to_csv('data/historical_stock_data_final.csv')

print("All data downloaded and saved.")


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


All data downloaded and saved.


# FEATURE ENGINEERING


### TECHNICAL INDICATORS:

* MACD (Moving Average Convergence Divergence)
* RSI (Relative Strength Index)
* Bollinger Bands High (the low is commented out)
* ADX (Average Directional Index)
* ATR (Average True Range)
* CCI (Commodity Channel Index)
* MFI (Money Flow Index)
* Stochastic Oscillator
* Keltner Channel
* VWAP (Volume Weighted Average Price)
* EOM (Ease Of Movement)
* VPT (Volume Price Trend)

In [4]:
# 1. Load data
data_yf = pd.read_csv('data/historical_stock_data_final.csv')

In [5]:
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2005-01-01,26.799999,27.100000,26.620001,26.670000,18.611694,392522300.0,MSFT
1,2005-01-08,26.600000,26.860001,26.040001,26.120001,18.227879,390071600.0,MSFT
2,2005-01-15,26.030001,26.350000,25.639999,25.650000,17.899891,262141600.0,MSFT
3,2005-01-22,25.760000,26.650000,25.639999,26.180000,18.269747,405236700.0,MSFT
4,2005-01-29,26.350000,26.520000,26.100000,26.320000,18.367455,332545200.0,MSFT
...,...,...,...,...,...,...,...,...
457157,2023-11-27,22.910000,23.530001,22.780001,23.150000,23.065542,5837000.0,NWS
457158,2023-12-04,23.059999,23.469999,22.209999,22.940001,22.856310,3872900.0,NWS
457159,2023-12-11,22.950001,24.600000,22.850000,24.530001,24.440508,6877000.0,NWS
457160,2023-12-18,24.570000,25.020000,24.320000,24.940001,24.849012,3202000.0,NWS


In [6]:
# 2. Clean data_yf: DROP NAs
data_yf.dropna(inplace=True)

# 3. Add technical indicators

data_yf['macd'] = trend.MACD(data_yf['Close']).macd()
data_yf['macd_signal'] = trend.MACD(data_yf['Close']).macd_signal()
data_yf['rsi'] = momentum.RSIIndicator(data_yf['Close']).rsi()
data_yf['bb_high'] = volatility.BollingerBands(data_yf['Close']).bollinger_hband()
#data_yf['bb_low'] = volatility.BollingerBands(data_yf['Close']).bollinger_lband()
data_yf['adx'] = trend.ADXIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).adx()
data_yf['atr'] = volatility.AverageTrueRange(data_yf['High'], data_yf['Low'], data_yf['Close']).average_true_range()
data_yf['cci'] = trend.CCIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).cci()
data_yf['mfi'] = volume.MFIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).money_flow_index()
data_yf['so'] = momentum.StochasticOscillator(data_yf['High'], data_yf['Low'], data_yf['Close']).stoch()
# Calculate additional indicators
data_yf['kc'] = volatility.KeltnerChannel(data_yf['High'], data_yf['Low'], data_yf['Close']).keltner_channel_mband()
data_yf['vwap'] = volume.VolumeWeightedAveragePrice(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).volume_weighted_average_price()
data_yf['eom'] = volume.EaseOfMovementIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).ease_of_movement()
data_yf['vpt'] = volume.VolumePriceTrendIndicator(data_yf['Close'], data_yf['Volume']).volume_price_trend()

In [7]:
data_yf.columns


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker',
       'macd', 'macd_signal', 'rsi', 'bb_high', 'adx', 'atr', 'cci', 'mfi',
       'so', 'kc', 'vwap', 'eom', 'vpt'],
      dtype='object')

In [8]:
data_yf.shape

(457162, 21)

#### MOVING AVERAGES

In [9]:
# Create lagged features for each indicator for the previous 5 days
indicator_cols = [col for col in data_yf.columns if col not in ['win', 'gain_loss_pct', 'Open', 'High', 'Low', 'Close', 'Volume','Adj Close']]
# Only including the moving average of 4 weeks and 12 weeks since this is long term trading
lags_to_include = [4, 12]

for col in indicator_cols:
    for i in lags_to_include:
        data_yf[f'{col}_lag_{i}'] = data_yf[col].shift(i)

# Remove rows with NaN values after shifting
data_yf.dropna(inplace=True)


In [10]:
# 4. Calculate gain/loss percentage and assign -1, 0, 1
data_yf['gain_loss_pct'] = (data_yf['Close'] - data_yf['Open']) / data_yf['Open']
data_yf['win'] = np.where(data_yf['gain_loss_pct'] > 0.005, 1, 
                          np.where(data_yf['gain_loss_pct'] < -0.0025, -1, 0)) # 1 if 0.5% gain, -1 if 0.25% loss, 0 otherwise
data_yf['Date'] = pd.to_datetime(data_yf['Date'])


In [11]:
# 5. Remove stocks that have not existed before 01/01/2005
first_appearance_dates = data_yf.groupby('Ticker')['Date'].min().sort_values(ascending=False)
first_appearance_dates

Ticker
VLTO   2023-10-02
KVUE   2023-05-01
GEHC   2022-12-12
CEG    2022-01-17
ABNB   2020-12-07
          ...    
F      2005-01-01
EXR    2005-01-01
EXPD   2005-01-01
EXC    2005-01-01
ISRG   2005-01-01
Name: Date, Length: 499, dtype: datetime64[ns]

In [12]:
#check when the stocks first appeared
first_appearance_dates = pd.to_datetime(first_appearance_dates)

#cutoff date 2005-01-01
cutoff_date = pd.Timestamp('2005-01-01')

#Only keeop stocks that have appeared on/before 2005-01-01
valid_tickers = first_appearance_dates[first_appearance_dates <= cutoff_date].index

data_yf = data_yf[data_yf['Ticker'].isin(valid_tickers)]

In [13]:
data_yf.to_csv('data/raw_data_yf.csv', index=False)

In [14]:
from sklearn.preprocessing import StandardScaler

float_columns = data_yf.select_dtypes(include=['float64']).columns

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize only the float columns
data_yf[float_columns] = scaler.fit_transform(data_yf[float_columns])

/var/folders/l3/h56ll3s97r3bp825gbh1f4q80000gq/T/ipykernel_24217/615959837.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_yf[float_columns] = scaler.fit_transform(data_yf[float_columns])


In [15]:
# create csv with features data
data_yf.to_csv('data/all_stock_data.csv', index=False)